In [16]:
import sys
print(sys.version)

3.7.5 (default, Nov  7 2019, 10:50:52) 
[GCC 8.3.0]


In [17]:
import pybgpstream as pbs
import time
from datetime import datetime
from collections import defaultdict
from typing import Dict, Set
import pickle


# RouteViews AMS-IX
#collectors = ["route-views.amsix"]
# RIPE RIS AmS-IX
collectors = ["rrc03"]

In [18]:
# Find a time where rib dumps occurred in this range
startSearchTime = "2020-01-07 00:00:00 UTC"
endSearchTime = "2020-01-07 23:59:59 UTC"

def getFirstRibDumpTime():
    fullDayRibStream = pbs.BGPStream(
        from_time=startSearchTime,
        until_time=endSearchTime,
        collectors=collectors,
        record_type="ribs"
    )
    for rec in fullDayRibStream.records():
        for elem in rec:
            if 'next-hop' in elem.fields:
                return rec.time
            
startTime = int(getFirstRibDumpTime())
endTimeShort = startTime + 60
endTimeLong = startTime + 60 * 15
print("Found RIB dumps at time %d" % startTime)

Found RIB dumps at time 1578355200


In [19]:
ribStream = pbs.BGPStream(
        collectors=collectors,
        record_type="ribs"
    )
ribStream.add_interval_filter(startTime, endTimeShort)

updateStream = pbs.BGPStream(
        collectors=collectors,
        record_type="updates"
    )
updateStream.add_interval_filter(startTime, endTimeLong)

In [20]:
def extractMatrix(stream):
    prefixSets = defaultdict(set)
    for rec in stream.records():
        for elem in rec:
            # if the record is a route, it will have 'next-hop' in the fields
            if 'next-hop' in elem.fields:

                prefix = elem.fields['prefix']

                announcer = elem.fields['as-path'].split(" ")[0]

                prefixSets[prefix].add(announcer)

    return prefixSets
        
matrix = extractMatrix(ribStream)

In [21]:
print("%d rows in matrix" % len(matrix))

ASes = set()
for row in matrix.values():
    ASes.update(row)
print("%d columns total" % len(ASes))

print("First 100 rows:")
for prefix, ases in list(matrix.items())[:10]:
    print(prefix, str(list(ases)))

934251 rows in matrix
77 columns total
First 100 rows:
0.0.0.0/0 ['57111', '47692', '43376', '205206', '64271', '51088', '50673']
::/0 ['20495', '47692', '6453', '205206', '43376', '50763', '51088', '50673']
1.0.0.0/24 ['8283', '60501', '57111', '8455', '12859', '1140', '1103', '15435', '6453', '205206', '43376', '64271', '51088', '57695', '29467', '8218', '30132', '50763', '24875', '50673', '59605', '47692', '14907', '47147']
100::6:6:6/128 ['50673']
1.0.4.0/22 ['8283', '60501', '57111', '8455', '12859', '1140', '1103', '15435', '6453', '205206', '43376', '64271', '51088', '57695', '6939', '29467', '8218', '30132', '50763', '24875', '50673', '59605', '47692', '14907', '20562', '47147']
1.0.4.0/24 ['8283', '60501', '57111', '8455', '12859', '1140', '1103', '15435', '6453', '205206', '43376', '64271', '51088', '57695', '6939', '29467', '8218', '30132', '50763', '24875', '50673', '59605', '47692', '14907', '20562', '47147']
1.0.5.0/24 ['8283', '60501', '57111', '8455', '12859', '1140', '

In [23]:
def removeSubsets(supersets):
    """ Removes all subsets from a list of (super)sets.
    """
    # deduplicate
    supersets = set(frozenset(row) for row in supersets)
    final_answer = []
    # defensive copy
    supersets = [set(superset) for superset in supersets]
    supersets.sort(key=len, reverse=True)
    i = 0
    while i < len(supersets):
        final_answer.append(supersets[i])

        for j in reversed(range(i+1, len(supersets))):
            if supersets[j].issubset(supersets[i]):
                del supersets[j]

        i += 1

    return final_answer

clusters = list(matrix.values())
clusters = removeSubsets(clusters)
print(len(clusters))

97


In [24]:
for cluster in clusters:
    print(cluster)

{'8283', '60501', '57111', '8455', '12859', '1140', '8560', '1103', '15435', '6453', '205206', '43376', '64271', '8220', '51088', '37721', '57695', '6939', '29467', '6461', '8218', '30132', '50763', '24875', '50673', '59605', '6774', '6830', '47692', '12779', '12989', '14907', '20562', '47147'}
{'8283', '60501', '57111', '8455', '12859', '1140', '1103', '15435', '6453', '205206', '43376', '64271', '8220', '21320', '51088', '37721', '3292', '57695', '6939', '29467', '6461', '8218', '30132', '50763', '24875', '50673', '59605', '47692', '14907', '20562', '12956', '47147'}
{'8283', '60501', '57111', '8455', '12859', '1140', '8560', '1103', '15435', '6453', '205206', '43376', '64271', '21320', '51088', '37721', '57695', '6939', '29467', '6461', '8218', '30132', '50763', '24875', '50673', '59605', '47692', '14907', '20562', '5413', '12956', '47147'}
{'51088', '8283', '57695', '6939', '29467', '8218', '30132', '50763', '60501', '24875', '57111', '8455', '12859', '50673', '1140', '59605', '110

In [25]:
def extractUpdates(stream):
    updates = []
    for rec in stream.records():
        for elem in rec:
            if len(elem.fields) > 0:
                if elem.type in ['A', 'W']:
                    announcer = elem.fields.get('as-path', None)
                    if announcer is None:
                        announcer = elem.peer_asn
                    else:
                        announcer = announcer.split()[0]
                    updateType = '+' if elem.type == 'A' else '-'
                    updateTuple = (rec.time, announcer, updateType, elem.fields['prefix'])
                    updates.append(updateTuple)
                #print(elem.type)
                #print(elem.fields)
    return updates
updates = extractUpdates(updateStream)

In [26]:
print("%d updates scraped." % len(updates))
print("First 100 updates:")
for update in updates[:10]:
    print(update)

521349 updates scraped.
First 100 updates:
(1578355200.0, '1140', '+', '64.68.236.0/22')
(1578355200.0, '8218', '+', '170.0.108.0/22')
(1578355200.0, '8218', '+', '190.124.176.0/20')
(1578355200.0, '8218', '+', '190.124.190.0/24')
(1578355200.0, '8218', '+', '177.155.112.0/20')
(1578355200.0, '8218', '+', '131.221.138.0/24')
(1578355200.0, '8218', '+', '131.221.139.0/24')
(1578355200.0, '8218', '+', '131.221.136.0/24')
(1578355200.0, '8218', '+', '131.221.137.0/24')
(1578355200.0, '8218', '+', '177.155.112.0/20')


In [ ]:
pickle_filename = "rrc03_matrix_and_updates.pickle"

In [27]:
print("Saving matrix and updates to pickle", pickle_filename)

obj = {'matrix': matrix, 'updates':updates}
with open(pickle_filename, 'wb') as fp:
    pickle.dump(obj, fp)
print("Done.")

Saving matrix and updates to pickle rrc03_matrix_and_updates.pickle
Done.


In [43]:
print("Loading pickle", pickle_filename)
with open(pickle_filename, 'rb') as fp:
    obj = pickle.load(fp)
print("Done.")
matrix = obj['matrix']
updates = obj['updates']

Loading pickle rrc03_matrix_and_updates.pickle
Done.


In [44]:
clusters = [set(row) for row in matrix.values()]
clusters = list(removeSubsets(clusters))

def closestMatchingCluster(newRow):
    """ Which cluster most contains the new row?
    """
    newRow = set(newRow)
    bestIndex = -1
    smallestDiff = len(clusters) # some arbitrary huge number to act as infinity
    for i, cluster in enumerate(clusters):
        diff = len([x for x in newRow if x not in cluster])
        if diff < smallestDiff:
            smallestDiff = diff
            bestIndex = i
    if smallestDiff != 0:
        print("A growth has to occur!")
    return bestIndex

tagChangesPerEpoch = defaultdict(int)
ruleChangesPerEpoch = defaultdict(int)

epochs = set()
prefixes = set()

for time, sourceAS, updateType, prefix in updates:
    time = int(time)
    epochs.add(time)
    prefixes.add(prefix)
    row = matrix.setdefault(prefix, set())
    if updateType == "-":
        if sourceAS in row:
            tagChangesPerEpoch[time] += 1
        row.discard(sourceAS)
    else: # '+'
        if sourceAS in row:
            continue
        tagChangesPerEpoch[time] += 1
        row.add(sourceAS)
        
        bestClusterIdx = closestMatchingCluster(row)
        bestCluster = clusters[bestClusterIdx]
        if row.issubset(bestCluster):
            continue
        oldClusterSize = len(bestCluster)
        bestCluster.update(row)
        newClusterSize = len(bestCluster)
        ruleChangesPerEpoch[time] += (newClusterSize - oldClusterSize)
        

A growth has to occur!


In [45]:
print(len(epochs)) # this  better be 15 minutes (900 sec)
print(len(prefixes)) # this better be big

901
148239


In [49]:
from collections import Counter

print("Tag changes per epoch:")
print(dict(Counter(tagChangesPerEpoch.values())))
print("Avg tag changes:", sum(tagChangesPerEpoch.values())/len(tagChangesPerEpoch))
    
    

Tag changes per epoch:
{11: 5, 512: 1, 139: 1, 162: 1, 24: 1, 8: 19, 124: 1, 44: 1, 86: 1, 6: 14, 4: 23, 55: 2, 46: 1, 1: 118, 2: 41, 3: 37, 39: 6, 47: 1, 42: 1, 27: 3, 56: 1, 84: 1, 26: 2, 19: 1, 7: 11, 23: 2, 9: 7, 5: 12, 58: 1, 16: 6, 10: 15, 12: 7, 20: 2, 22: 3, 15: 3, 63: 1, 34: 2, 21: 3, 14: 3, 13: 3, 30: 1, 67: 2, 82: 1, 18: 2, 17: 2, 99: 1, 265: 1, 466: 1, 119: 1, 37: 1, 360: 1, 32: 1, 126: 1, 614: 1, 134: 1}
Avg tag changes: 16.18848167539267


In [50]:
print("Rule changes per epoch:")
print(dict(Counter(ruleChangesPerEpoch.values())))

Rule changes per epoch:
{1: 1}


In [29]:
print(clusters[0])

{'8283', '60501', '57111', '8455', '12859', '1140', '8560', '1103', '15435', '6453', '205206', '43376', '64271', '8220', '51088', '37721', '57695', '6939', '29467', '6461', '8218', '30132', '50763', '24875', '50673', '59605', '6774', '6830', '47692', '12779', '12989', '14907', '20562', '47147'}
